In [1]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd 
from path import Path

In [2]:
file_path = Path("../Resources/charity_data.csv")
charity_df = pd.read_csv(file_path)
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


DATA CLEANING

Our target variable is IS_SUCCESSFUL column. Every other column is a feature. NAME and EIN columns will have no effect on potential outcomes so those columns need to be dropped. 

In [3]:
charity_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
cleaned_df = charity_df.drop(columns=["EIN", "NAME"])

In [5]:
cleaned_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
#classification and application_type need to be bucketed to remove some features. 
cleaned_df["CLASSIFICATION"].value_counts().head(10)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: CLASSIFICATION, dtype: int64

In [7]:
#bucket classification
classification_counts = cleaned_df["CLASSIFICATION"].value_counts()

replace_classification = list(classification_counts[classification_counts<700].index)

for classification in replace_classification:
    cleaned_df["CLASSIFICATION"] = cleaned_df["CLASSIFICATION"].replace(classification, "other")
    
cleaned_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [8]:
#bucket application type 
application_counts = cleaned_df["APPLICATION_TYPE"].value_counts()

replace_application = list(application_counts [application_counts <500].index)

for application in replace_application:
    cleaned_df["APPLICATION_TYPE"] = cleaned_df["APPLICATION_TYPE"].replace(application, "other")
cleaned_df["APPLICATION_TYPE"].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
#encode the rest of the results via OneHotEncoder

#get categorical results 
categorical_vars = cleaned_df.dtypes[cleaned_df.dtypes=="object"].index.tolist()

#create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

#fit and transform 
encode_df = pd.DataFrame(enc.fit_transform(cleaned_df[categorical_vars]))

#rename encoded columns
encode_df.columns = enc.get_feature_names(categorical_vars)
encode_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_other,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
#merge encoded_df with cleaned_df
merged_df = cleaned_df.merge(encode_df, left_index=True, right_index=True).drop(categorical_vars,1)
merged_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Now that our data has been cleaned and encoded we can split it into training and testing sets, and scale it

In [12]:
#split data
y = merged_df["IS_SUCCESSFUL"].values
X = merged_df.drop(columns=["IS_SUCCESSFUL"]).values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

now that our data has been scaled its time to build our deep learning model

In [13]:
#define deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features*2
hidden_nodes_layer2 = number_input_features

nn = tf.keras.models.Sequential()

#add first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

#add second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#compile the model and customize metrics 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
#train the deep nn
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

#evaluate the model 
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 5s 208us/sample - loss: 0.5675 - accuracy: 0.7213
Epoch 2/50
25724/25724 [==============================] - 3s 136us/sample - loss: 0.5538 - accuracy: 0.7284
Epoch 3/50
25724/25724 [==============================] - 4s 147us/sample - loss: 0.5517 - accuracy: 0.7293
Epoch 4/50
25724/25724 [==============================] - 4s 145us/sample - loss: 0.5495 - accuracy: 0.7314
Epoch 5/50
25724/25724 [==============================] - 4s 144us/sample - loss: 0.5483 - accuracy: 0.7310
Epoch 6/50
25724/25724 [==============================] - 4s 155us/sample - loss: 0.5469 - accuracy: 0.7328
Epoch 7/50
25724/25724 [==============================] - 4s 148us/sample - loss: 0.5464 - accuracy: 0.7324
Epoch 8/50
25724/25724 [==============================] - 4s 169us/sample - loss: 0.5458 - accuracy: 0.7319
Epoch 9/50
25724/25724 [==============================] - 4s 143us/sample - loss: 0.5460 - accuracy: 0.7338
Epoch

First attempt with two hidden layers resulted in an accuracy score of approximately 73%. We would like to achieve an accuracy level of 75%. In order to increase our accuracy score we will begin with increasing the number of neurons in our hidden layers 

In [15]:
#attempt 2 

#define deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features*2
hidden_nodes_layer2 = number_input_features*2

nn = tf.keras.models.Sequential()

#add first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

#add second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#compile the model and customize metrics 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
#train the deep nn
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

#evaluate the model 
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 5s 208us/sample - loss: 0.5673 - accuracy: 0.7215
Epoch 2/50
25724/25724 [==============================] - 5s 178us/sample - loss: 0.5553 - accuracy: 0.7284
Epoch 3/50
25724/25724 [==============================] - 5s 203us/sample - loss: 0.5524 - accuracy: 0.7286
Epoch 4/50
25724/25724 [==============================] - 5s 195us/sample - loss: 0.5502 - accuracy: 0.7298
Epoch 5/50
25724/25724 [==============================] - 4s 166us/sample - loss: 0.5491 - accuracy: 0.7316
Epoch 6/50
25724/25724 [==============================] - 6s 218us/sample - loss: 0.5480 - accuracy: 0.7314
Epoch 7/50
25724/25724 [==============================] - 5s 187us/sample - loss: 0.5474 - accuracy: 0.7320
Epoch 8/50
25724/25724 [==============================] - 4s 142us/sample - loss: 0.5462 - accuracy: 0.7330
Epoch 9/50
25724/25724 [==============================] - 4s 152us/sample - loss: 0.5455 - accuracy: 0.7335
Epoch

second attempt accuracy score was the same as the first attempt. Increasing the number of neurons within the hidden layers did not increase our accuracy score. It just took longer to get the results. For the third attempt we will increase the number of epochs and use the same parameters from attempt 1. 

In [17]:
#define deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features*2
hidden_nodes_layer2 = number_input_features

nn = tf.keras.models.Sequential()

#add first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

#add second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#compile the model and customize metrics 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
#train the deep nn
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

#evaluate the model 
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 5s 191us/sample - loss: 0.5709 - accuracy: 0.7219
Epoch 2/100
25724/25724 [==============================] - 4s 139us/sample - loss: 0.5554 - accuracy: 0.7287
Epoch 3/100
25724/25724 [==============================] - 4s 159us/sample - loss: 0.5517 - accuracy: 0.7298
Epoch 4/100
25724/25724 [==============================] - 4s 137us/sample - loss: 0.5499 - accuracy: 0.7308
Epoch 5/100
25724/25724 [==============================] - 4s 137us/sample - loss: 0.5487 - accuracy: 0.7310
Epoch 6/100
25724/25724 [==============================] - 4s 139us/sample - loss: 0.5476 - accuracy: 0.7307
Epoch 7/100
25724/25724 [==============================] - 4s 138us/sample - loss: 0.5467 - accuracy: 0.7319
Epoch 8/100
25724/25724 [==============================] - 4s 141us/sample - loss: 0.5462 - accuracy: 0.7337
Epoch 9/100
25724/25724 [==============================] - 4s 138us/sample - loss: 0.5453 - accuracy: 0.7

increasing the number of epochs has had the best result on accuracy so far. Although the total accuracy score did not break 74% we can see that many of the invididual epochs did indeed break 74% accuracy. More attempts could be made, but a more sophisticated model is likely needed to break the 75% threshold. 